# Data Cleaning

## Introduction

This notebook goes through the steps taken with the data collected in order to get cleaned organized data in two standard text formats. The notebook will contain the nexts steps.

1. **Cleaning the data -** I will use text pre-procesing techniques to get the dta into shape.
2. **Organizing the data -** I'l organize the data into a way that is easy to input into other algoithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of texts
2. **Document-Term Matrix** - words counts in matrix format

### Problem Statement

My goal is to look look a the latest headlines of the main newspapers in Perú and note simmilarities and differences.

## Getting the data

The data was taken from the [Twitter API](https://developer.twitter.com) and the newspaper where selected according to a list found in [Diarios de Perú](http://www.diariosdeperu.com.pe) as well as annecdotal experience. The journals to investigate:

| Newspaper | Twitter handle |
| ----------- | ----------- |
| El Comercio | elcomercio_peru |
| La República | larepublica_pe |
| Perú 21 | peru21noticias |
| Trome | tromepe |
| Gestión | Gestionpe |
| Diario Correo | diariocorreo |
| Diario Expreso | ExpresoPeru |
| Diario Ojo | diarioojo |
| Diario El Peruano | DiarioElPeruano |
| Diario La Razón | larazon_pe |

In [7]:
import json
import os
import re
import requests

from urllib.parse import urlencode
from dotenv import load_dotenv

load_dotenv()

BASE_DIR = os.environ.get("BASE_DIR")
BEARER_TOKEN = os.environ.get("BEARER_TOKEN")

### Getting the user ids

In [30]:
newspapers = [
    "elcomercio_peru",
    "larepublica_pe",
    "peru21noticias",
    "tromepe",
    "Gestionpe",
    "diariocorreo",
    "ExpresoPeru",
    "diarioojo",
    "DiarioElPeruano",
    "larazon_pe"
]
headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}

In [31]:
newspapers_id = {}

for newspaper in newspapers:
    username_url = f"https://api.twitter.com/2/users/by/username/{newspaper}"
    response = requests.get(username_url, headers=headers)

    newspapers_id[newspaper] = response.json()["data"]["id"]

with open(f'{BASE_DIR}/data/raw/newspapers_id.json', 'w') as write_file:
    json.dump(newspapers_id, write_file)

### Getting tweets from newspapers

In [2]:
with open(f'{BASE_DIR}/data/raw/newspapers_id.json', 'r') as read_file:
    newspapers_id = json.load(read_file)

I'm choosing start time and end time of tweets in order to be able to select tweets from different times and be actively collecting tweets as the months go.

In [33]:
for newspaper, newspaper_id in newspapers_id.items():
    query = {
    "max_results": 100,
    "tweet.fields": "id,text,created_at,public_metrics,possibly_sensitive,referenced_tweets",
    "start_time": "2020-12-01T00:00:00Z",
    "end_time": "2021-01-15T00:00:00Z",
    }
    payload = urlencode(query, safe=",:")
    
    tweets_url = f"https://api.twitter.com/2/users/{newspaper_id}/tweets"
    response = requests.get(tweets_url, headers=headers, params=payload)
    print(f"{newspaper}: status code {response.status_code}")

    try:
        response_data = response.json()["data"] # List of tweets
        response_meta = response.json()["meta"]

        next_token = response_meta["next_token"]
        query["pagination_token"] = next_token
        payload = urlencode(query, safe=",")
    except KeyError:
        continue

    while True:
        new_response = requests.get(tweets_url, headers=headers, params=payload)

        response_data += new_response.json()["data"]

        response_meta = new_response.json()["meta"]

        try:
            next_token = response_meta["next_token"]
            query["pagination_token"] = next_token
            payload = urlencode(query, safe=",:")
        except KeyError:
            with open(f"{BASE_DIR}/data/raw/data_{newspaper}.json", "w") as write_file:
                json.dump({"data": response_data}, write_file)
            break

    with open(f"{BASE_DIR}/data/raw/data_{newspaper}.json", "w") as write_file:
        json.dump({"data": response_data}, write_file)

elcomercio_peru: status code 200
larepublica_pe: status code 200
peru21noticias: status code 200
tromepe: status code 200
Gestionpe: status code 200
diariocorreo: status code 200
ExpresoPeru: status code 200
diarioojo: status code 200
DiarioElPeruano: status code 200
larazon_pe: status code 200


## Cleaning the data

Since most of the data we are dealing with is text data, I'm goig to be using some common text pre-processing techniques.

For that I'm going to follow the MVP __(Minimum Viable Product)__ approach. For that the main resource I'll be using is a talk from PyOhio by [Alice Zhao](https://github.com/adashofdata/nlp-in-python-tutorial/blob/master/1-Data-Cleaning.ipynb). The cleaning steps I'll be taking are.

**Removing tweets that are outside of the scope**

* Tweets corresponding to the cover page announcement
* Tweets corresponding to caricature of the day
* Tweets corresponding to the horoscope
* ...

**Common data cleaning steps on all text:**

* Make text all lowercase
* Remove punctuation
* Remove numerical values
* Remove common non.sensical text (\n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**

* Stemming/lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos

In [8]:
import pandas as pd
import numpy as np
import pickle as pk

pd.options.display.max_colwidth = 300
pd.options.display.max_rows = 25

In [4]:
newspaper_df = {}

for newspaper, newspaper_id in newspapers_id.items():
    with open(f'{BASE_DIR}/data/raw/data_{newspaper}.json', 'r') as read_file:
        json_file = json.load(read_file)

    newspaper_df[newspaper] = pd.json_normalize(json_file["data"])
    newspaper_df[newspaper]["newspaper"] = newspaper

In [5]:
with open(f'{BASE_DIR}/data/interim/newspaper_df.pkl', 'wb') as write_file:
    pk.dump(newspaper_df, write_file)

In [6]:
data_raw = pd.concat(newspaper_df.values())

In [7]:
data_raw["created_at"] = pd.to_datetime(data_raw["created_at"], infer_datetime_format=True, errors='coerce')

In [8]:
data_raw.head()

,created_at,possibly_sensitive,text,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets,newspaper
0,2021-01-14 23:58:50+00:00,False,Keiko Fujimori: “Me llena de ilusión y energía haber empezado a caminar para llegar a todo nuestro país” https://t.co/m8E0YzsXIl,1349868608600608768,6,101,47,6,NaN,elcomercio_peru
1,2021-01-14 23:55:38+00:00,False,Línea 1 del Metro de Lima: nuevo horario ante ampliación del toque de queda https://t.co/a3Mqp1a5Yd,1349867803319402499,1,0,4,0,NaN,elcomercio_peru
2,2021-01-14 23:52:31+00:00,False,La policía y los militares de EE.UU. se enfrentan a un enemigos internos https://t.co/PwT8AFUHbG,1349867020121214978,2,0,2,0,NaN,elcomercio_peru
3,2021-01-14 23:49:25+00:00,False,Autos antiguos en las calles limeñas https://t.co/AOjXuLhzYD,1349866239481556992,2,0,11,0,NaN,elcomercio_peru
4,2021-01-14 23:46:13+00:00,False,Sporting Cristal oficializó el regreso de Jesús Pretell para la temporada 2021 https://t.co/aQ61CkkgsM,1349865436259774466,0,0,5,0,NaN,elcomercio_peru


In [9]:
data_raw.tail()

,created_at,possibly_sensitive,text,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets,newspaper
1203,2020-12-01 04:42:36+00:00,False,"Cerca de la #Navidad, el Instituto Nacional de #Salud recomienda #juguetes que incentiven la actividad familiar, de acuerdo a la edad de los pequeños. Click aquí: https://t.co/jkmJzHunNw",1333632569729044480,0,0,3,0,NaN,larazon_pe
1204,2020-12-01 03:59:32+00:00,False,Los #turistas nacionales podrán acceder a la tarifa de S/ 12 soles por tramo a partir del 1 de diciembre y en 4 horarios de atención.\n\nAquí la nota: https://t.co/ODiJ92dH3b,1333621728984133632,1,0,6,0,NaN,larazon_pe
1205,2020-12-01 03:46:39+00:00,False,"El Instituto Peruano de Paternidad Responsable (#Inppares ), promueve #campaña totalmente gratuita dual de detección de VIH/Sífilis y luego acceder a una consejería. Click aquí: https://t.co/ZKUh0UEuqs",1333618485482622978,0,0,2,0,NaN,larazon_pe
1206,2020-12-01 03:05:43+00:00,False,El Ministerio del Interior (#Mininter ) descartó la posibilidad sobre la desactivación del Grupo #Terna. Click aquí: https://t.co/qSRjHSzIEE,1333608187010965504,1,0,0,0,NaN,larazon_pe
1207,2020-12-01 00:25:12+00:00,False,El expresidente #OllantaHumala señaló que llama la atención que Vizcarra postule por un partido que lo vacó.\n\nNota completa aquí: https://t.co/QSbj2EEIEa,1333567792386289666,0,1,1,1,NaN,larazon_pe


In [10]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18632 entries, 0 to 1207
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype              
---  ------                        --------------  -----              
 0   created_at                    18632 non-null  datetime64[ns, UTC]
 1   possibly_sensitive            18632 non-null  bool               
 2   text                          18632 non-null  object             
 3   id                            18632 non-null  object             
 4   public_metrics.retweet_count  18632 non-null  int64              
 5   public_metrics.reply_count    18632 non-null  int64              
 6   public_metrics.like_count     18632 non-null  int64              
 7   public_metrics.quote_count    18632 non-null  int64              
 8   referenced_tweets             875 non-null    object             
 9   newspaper                     18632 non-null  object             
dtypes: bool(1), datetime64[ns, UTC](1),

From the first looks, as well a the look on the head and tail I get to see how many of the variables are categorical and how many are numerical. Also, I only found missing values in the referenced tweets field. So, first for the numerical value I'm going to take a look of some metrics before the EDA.

**Numerical variables:**

1. `created_at`: Timestamp of the tweet
2. `public_metrics.retweet_count`: Number of times a tweet was retweeted
3. `public_metrics.like_count`: Number of likes a tweet has
4. `public_metrics.quote_count`: Number of times a tweet was quoted

**Categorical variables**

1. `id`: Unique identifier of tweet
2. `positively_sensitive`: Boolean variable of whether a tweet might contain sensitive information
3. `text`: Actual text of the tweet
4. `referenced_tweet`: Whether this tweet is a retweet or a quoted tweet
5. `newspaper`: Twitter handle if the newspaper the tweet belongs to

In [11]:
data_raw["newspaper"].value_counts()

ExpresoPeru        2562
diarioojo          2373
Gestionpe          2370
diariocorreo       2206
tromepe            2085
DiarioElPeruano    1957
larepublica_pe     1660
peru21noticias     1253
larazon_pe         1208
elcomercio_peru     958
Name: newspaper, dtype: int64

From the newspaper value counts I found that Expreso more than doubles the ammount of tweets of El Comercio and La Razón. Also, I look for the journals that respond more in the form of *referenced tweets*

In [12]:
data_raw["newspaper"][data_raw["referenced_tweets"].notna()].value_counts()

larepublica_pe     661
DiarioElPeruano    160
elcomercio_peru     26
ExpresoPeru         23
Gestionpe            3
peru21noticias       2
Name: newspaper, dtype: int64

The journal that retweets, or replies more is **La República**, which is not the one that tweets more, and surprisingly **Expresso** which is the one that tweets the most has barely any retweets.

What I found for **La República** is that most of his retweets come from their accounts for sports and politics, and that was supported by the ammount of times that their account names occur, when looking at their most user words.

In [13]:
data_raw[data_raw["possibly_sensitive"] == True]

,created_at,possibly_sensitive,text,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets,newspaper
624,2021-01-11 19:04:32+00:00,True,El MEF determinó la información que las entidades financieras deberán entregar a Sunat para combatir la elusión y evasión tributaria.\n\nMás información►https://t.co/aoOjNDW1ef https://t.co/Qet8hKOT4D,1348707383187484677,6,1,10,0,NaN,larepublica_pe


The only possitively sensitive tweets comes from **La Repúbica** and was posted on January 2021. More analysis for the other variables will be done during the EDA process.

With that said, I'll begin by removing the posts which are not headlines, as specified at the beginning.

In [17]:
data = data_raw

data.drop(data[data["text"].str.contains('horóscopo diario', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('horóscopo de', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('horóscopo hoy', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('horóscopo y tarot', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('Buenos días', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('la caricatura de', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('las caricaturas de', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('portada impresa', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('trome gol', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('trome gol', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('no te pierdas las chiquitas de hoy', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('esta es la portada', flags=re.IGNORECASE, regex=True)].index, inplace=True)

In [20]:
data.newspaper.value_counts()

ExpresoPeru        2225
diarioojo          2039
Gestionpe          2036
diariocorreo       1884
tromepe            1775
DiarioElPeruano    1660
larepublica_pe     1403
peru21noticias     1043
larazon_pe         1004
elcomercio_peru     798
Name: newspaper, dtype: int64

In [21]:
import re
import string

def clean_text_first_pass(text):
    """Get rid of other punctuation and non-sensical text identified.

    Args:
        text (string): text to be processed.
    """
    text = text.lower()
    text = re.sub(r'^http[s]?:\/\/.*[\r\n]*', ' ', text)
    text = re.sub(r'^bit.ly', ' ', text)
    text = re.sub(r'^dozz.es', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[‘’“”…«»►¿¡|│]', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', ' ', text)

    return text

first_pass = lambda x: clean_text_first_pass(x)

In [22]:
data["text_clean"] = data.text.apply(first_pass)
data["text_clean"]

0                                                                               keiko fujimori me llena de ilusión y energía haber empezado a caminar para llegar a todo nuestro país 
1                                                                                                           línea  del metro de lima nuevo horario ante ampliación del toque de queda 
2                                                                                                              la policía y los militares de eeuu se enfrentan a un enemigos internos 
3                                                                                                                             autos  antiguos en las calles limeñas httpstcoaojxulhzyd
4                                                                                                          sporting cristal oficializó el regreso de jesús pretell para la temporada  
                                                                                     

In [23]:
data[data["possibly_sensitive"] == True]

,created_at,possibly_sensitive,text,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets,newspaper,text_clean


In [24]:
data.rename(lambda x: x.replace('public_metrics.', ''), axis='columns', inplace=True)
data.head()

,created_at,possibly_sensitive,text,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,text_clean
0,2021-01-14 23:58:50+00:00,False,Keiko Fujimori: “Me llena de ilusión y energía haber empezado a caminar para llegar a todo nuestro país” https://t.co/m8E0YzsXIl,1349868608600608768,6,101,47,6,NaN,elcomercio_peru,keiko fujimori me llena de ilusión y energía haber empezado a caminar para llegar a todo nuestro país
1,2021-01-14 23:55:38+00:00,False,Línea 1 del Metro de Lima: nuevo horario ante ampliación del toque de queda https://t.co/a3Mqp1a5Yd,1349867803319402499,1,0,4,0,NaN,elcomercio_peru,línea del metro de lima nuevo horario ante ampliación del toque de queda
2,2021-01-14 23:52:31+00:00,False,La policía y los militares de EE.UU. se enfrentan a un enemigos internos https://t.co/PwT8AFUHbG,1349867020121214978,2,0,2,0,NaN,elcomercio_peru,la policía y los militares de eeuu se enfrentan a un enemigos internos
3,2021-01-14 23:49:25+00:00,False,Autos antiguos en las calles limeñas https://t.co/AOjXuLhzYD,1349866239481556992,2,0,11,0,NaN,elcomercio_peru,autos antiguos en las calles limeñas httpstcoaojxulhzyd
4,2021-01-14 23:46:13+00:00,False,Sporting Cristal oficializó el regreso de Jesús Pretell para la temporada 2021 https://t.co/aQ61CkkgsM,1349865436259774466,0,0,5,0,NaN,elcomercio_peru,sporting cristal oficializó el regreso de jesús pretell para la temporada


From checking the resulting text I found that there are some tweets that contain emojis that haven't been removed. For that I will use the `demoji` package.

In [25]:
import demoji

replace_emojis = lambda x: demoji.replace(x, "")

In [26]:
data["text_clean"] = data["text_clean"].apply(replace_emojis)
data.head()

,created_at,possibly_sensitive,text,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,text_clean
0,2021-01-14 23:58:50+00:00,False,Keiko Fujimori: “Me llena de ilusión y energía haber empezado a caminar para llegar a todo nuestro país” https://t.co/m8E0YzsXIl,1349868608600608768,6,101,47,6,NaN,elcomercio_peru,keiko fujimori me llena de ilusión y energía haber empezado a caminar para llegar a todo nuestro país
1,2021-01-14 23:55:38+00:00,False,Línea 1 del Metro de Lima: nuevo horario ante ampliación del toque de queda https://t.co/a3Mqp1a5Yd,1349867803319402499,1,0,4,0,NaN,elcomercio_peru,línea del metro de lima nuevo horario ante ampliación del toque de queda
2,2021-01-14 23:52:31+00:00,False,La policía y los militares de EE.UU. se enfrentan a un enemigos internos https://t.co/PwT8AFUHbG,1349867020121214978,2,0,2,0,NaN,elcomercio_peru,la policía y los militares de eeuu se enfrentan a un enemigos internos
3,2021-01-14 23:49:25+00:00,False,Autos antiguos en las calles limeñas https://t.co/AOjXuLhzYD,1349866239481556992,2,0,11,0,NaN,elcomercio_peru,autos antiguos en las calles limeñas httpstcoaojxulhzyd
4,2021-01-14 23:46:13+00:00,False,Sporting Cristal oficializó el regreso de Jesús Pretell para la temporada 2021 https://t.co/aQ61CkkgsM,1349865436259774466,0,0,5,0,NaN,elcomercio_peru,sporting cristal oficializó el regreso de jesús pretell para la temporada


Also, form redoing the analysis, and looking at the tweeter feeds from many of the newspapers I found that there are some patterns of writing that do not add to the content, like calls to action, that interact with the audience, but do not add any significance to the headline. I'll be checking at the twitter feed for such patterns and add them during cleaning. I'm not adding them as stopwords because these calls to action in specific are groups of words.

In [27]:
def clean_text_second_pass(text):
    """Get rid of other punctuation and non-sensical text identified.

    Args:
        text (string): text to be processed.
    """
    text = re.sub('click aquí', '', text)
    text = re.sub('lee aquí el blog de', '', text)
    text = re.sub('vía gestionpe', '', text)
    text = re.sub('entrevista exclusiva', '', text)
    text = re.sub('en vivo', '', text)
    text = re.sub('entérate más aquí', '', text)
    text = re.sub('lee la columna de', '', text)
    text = re.sub('lee y comenta', '', text)
    text = re.sub('lea hoy la columna de', '', text)
    text = re.sub('escrito por', '', text)
    text = re.sub('una nota de', '', text)
    text = re.sub('aquí la nota', '', text)
    text = re.sub('nota completa aquí', '', text)
    text = re.sub('lee más', '', text)

    text = re.sub('http[s]?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = re.sub('  ', ' ', text)

    return text

second_pass = lambda x: clean_text_second_pass(x)

In [29]:
data["text_clean"] = data.text_clean.apply(second_pass)
data.head()

,created_at,possibly_sensitive,text,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,text_clean
0,2021-01-14 23:58:50+00:00,False,Keiko Fujimori: “Me llena de ilusión y energía haber empezado a caminar para llegar a todo nuestro país” https://t.co/m8E0YzsXIl,1349868608600608768,6,101,47,6,NaN,elcomercio_peru,keiko fujimori me llena de ilusión energía haber empezado caminar para llegar todo nuestro país
1,2021-01-14 23:55:38+00:00,False,Línea 1 del Metro de Lima: nuevo horario ante ampliación del toque de queda https://t.co/a3Mqp1a5Yd,1349867803319402499,1,0,4,0,NaN,elcomercio_peru,línea del metro de lima nuevo horario ante ampliación del toque de queda
2,2021-01-14 23:52:31+00:00,False,La policía y los militares de EE.UU. se enfrentan a un enemigos internos https://t.co/PwT8AFUHbG,1349867020121214978,2,0,2,0,NaN,elcomercio_peru,la policía los militares de eeuu se enfrentan un enemigos internos
3,2021-01-14 23:49:25+00:00,False,Autos antiguos en las calles limeñas https://t.co/AOjXuLhzYD,1349866239481556992,2,0,11,0,NaN,elcomercio_peru,autos antiguos en las calles limeñas
4,2021-01-14 23:46:13+00:00,False,Sporting Cristal oficializó el regreso de Jesús Pretell para la temporada 2021 https://t.co/aQ61CkkgsM,1349865436259774466,0,0,5,0,NaN,elcomercio_peru,sporting cristal oficializó el regreso de jesús pretell para la temporada


In [30]:
data.to_pickle(f"{BASE_DIR}/data/interim/data_clean.pkl")

## Organizing the data

Now I need to get the data in both of the standar text formats:

1. **Corpus -** a collection of text
2. **Document-Term matrix -** word counts in matrix format

In the case of the tweets, I will start by combining all the clean texts and in the case of *Document-term matrix* tokenising the result.

In [31]:
df_clean = pd.read_pickle(f"{BASE_DIR}/data/interim/data_clean.pkl")

### Corpus

To create a corpus we will collapse all the existing rows and create another dataframe with the name of the newspaper and the tweets for analysis.

In [32]:
data_corpus = df_clean.groupby("newspaper")["text_clean"].apply(' '.join).reset_index()
data_corpus.head(15)

,newspaper,text_clean
0,DiarioElPeruano,conoce las actividades virtuales que se desarrollarán por el aniversario de lima será transmitido por facebook youtube ministra de defensa nuria esparch resalta la presencia cada vez mayor de mujeres en misiones de paz ops alerta que la variante del se expande en latinoamérica crisis política...
1,ExpresoPeru,giulliana loza keiko fujimori está realizando visitas al norte del país con la debida autorización judicial médicos intensivistas sostienen que lima metropolitana debió ser calificada como una región con un nivel de alerta muy alto partido entre aston villa vs everton fue aplazado por casos de...
2,Gestionpe,depósitos en dólares en soles qué convendrá más este año sinovac defiende su vacuna tras datos decepcionantes de ensayo en brasil línea del metro de lima establece nuevo horario ante ampliación del toque de queda en alemania temen que variante británica del covid pueda multiplicar por diez la...
3,diariocorreo,san borja minsa lanzan farmacia con servicio delivery para población vulnerable médicos de essalud extirpan tumor de un kilo salvan niño arequipeño de años video hombre de años murió después de cobrar su pensión en un banco de huancayo jne declara infundada apelación del apra para inscripci...
4,diarioojo,octogenario muere después de cobrar su pensión en un banco de junín luciana fuster celebra su cumpleaños en yate de marc anthony el invitado de honor video mujer acusada de asesinar su esposo él se apuñaló solo video gigi mitre enfrenta son tentación hay que ser un poquito más conscientes res...
5,elcomercio_peru,keiko fujimori me llena de ilusión energía haber empezado caminar para llegar todo nuestro país línea del metro de lima nuevo horario ante ampliación del toque de queda la policía los militares de eeuu se enfrentan un enemigos internos autos antiguos en las calles limeñas sporting cristal of...
6,larazon_pe,la premier violeta bermúdez dijo que se empezará una campaña informativa sobre la importancia de la vacunación ya que muchas personas tienen temor no creen el exministro de salud lamentó que la titular del minsa no pueda inducir al presidente estrategias mucho más claras juez del primer juzgad...
7,larepublica_pe,rt deporteslr sportboys anunció que seis miembros del plantel dieron positivo través de un comunicado rosados informaron rt deporteslr el puma carranza se refirió la marcha de hohberg sporting cristal el histórico capitán de la habló acerca del te onu el mundo se dirige hacia un calentamiento c...
8,peru21noticias,se registra mamá oso de anteojos su cría paseando por las áreas de machu picchu en cusco video arturo vidal se puso emotivo para despedir reinaldo rueda de la selección de chile george forsyth rechaza dinero de publicidad electoral para que estado lo use en el sistema de salud lambayeque gere...
9,tromepe,no le teme los nuevos sabores lambayeque fiscalía interviene jefe de comisaría incahuasi otros cuatro efectivos por presuntos sobornos el club estudiantil arrancó los trabajos en su sede pensando en la temporada alex valera jorge murrugarra los nuevos fichajes dijeron presente en la sesión go...


In [33]:
data_corpus = data_corpus.set_index('newspaper')

In [34]:
data_corpus.to_pickle(f'{BASE_DIR}/data/processed/corpus.pkl')

### Document-Term Matrix

From the corpus constructed in the step above I'll proceed to tokenizethe text to use with further techniques. For that I'll use scikit-learn's `CountVectorizer`, where every row represents a document and each column is a different row.

I'll also remove stop words.

In [4]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [10]:
def normalize_text(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop | t.is_space]

    return words

normalize = lambda x: normalize_text(x)

In [9]:
data_dtm = pd.read_pickle(f'{BASE_DIR}/data/processed/corpus.pkl')

In [11]:
data_dtm["doc"] = data_dtm["text_clean"].apply(normalize)
data_dtm["doc"]

newspaper
DiarioElPeruano    [conoce, actividades, virtuales, desarrollarán, aniversario, lima, transmitido, facebook, youtube, ministra, defensa, nuria, esparch, resalta, presencia, mujeres, misiones, paz, ops, alerta, variante, expande, latinoamérica, crisis, política, amenaza, futuro, imperio, económico, trump, debemos, ...
ExpresoPeru        [giulliana, loza, keiko, fujimori, realizando, visitas, norte, país, debida, autorización, judicial, médicos, intensivistas, sostienen, lima, metropolitana, debió, calificada, región, nivel, alerta, alto, partido, aston, villa, vs, everton, aplazado, casos, coronavirus, policía, judicial, buscar...
Gestionpe          [depósitos, dólares, soles, convendrá, año, sinovac, defiende, vacuna, datos, decepcionantes, ensayo, brasil, línea, metro, lima, establece, horario, ampliación, toque, queda, alemania, temen, variante, británica, covid, multiplicar, diez, incidencia, geely, foxconn, crean, alianza, fabricar, ve...
diariocorreo       [san, borja, m

In [13]:
data_dtm

,text_clean,doc
newspaper,,
DiarioElPeruano,conoce las actividades virtuales que se desarrollarán por el aniversario de lima será transmitido por facebook youtube ministra de defensa nuria esparch resalta la presencia cada vez mayor de mujeres en misiones de paz ops alerta que la variante del se expande en latinoamérica crisis política...,"[conoce, actividades, virtuales, desarrollarán, aniversario, lima, transmitido, facebook, youtube, ministra, defensa, nuria, esparch, resalta, presencia, mujeres, misiones, paz, ops, alerta, variante, expande, latinoamérica, crisis, política, amenaza, futuro, imperio, económico, trump, debemos, ..."
ExpresoPeru,giulliana loza keiko fujimori está realizando visitas al norte del país con la debida autorización judicial médicos intensivistas sostienen que lima metropolitana debió ser calificada como una región con un nivel de alerta muy alto partido entre aston villa vs everton fue aplazado por casos de...,"[giulliana, loza, keiko, fujimori, realizando, visitas, norte, país, debida, autorización, judicial, médicos, intensivistas, sostienen, lima, metropolitana, debió, calificada, región, nivel, alerta, alto, partido, aston, villa, vs, everton, aplazado, casos, coronavirus, policía, judicial, buscar..."
Gestionpe,depósitos en dólares en soles qué convendrá más este año sinovac defiende su vacuna tras datos decepcionantes de ensayo en brasil línea del metro de lima establece nuevo horario ante ampliación del toque de queda en alemania temen que variante británica del covid pueda multiplicar por diez la...,"[depósitos, dólares, soles, convendrá, año, sinovac, defiende, vacuna, datos, decepcionantes, ensayo, brasil, línea, metro, lima, establece, horario, ampliación, toque, queda, alemania, temen, variante, británica, covid, multiplicar, diez, incidencia, geely, foxconn, crean, alianza, fabricar, ve..."
diariocorreo,san borja minsa lanzan farmacia con servicio delivery para población vulnerable médicos de essalud extirpan tumor de un kilo salvan niño arequipeño de años video hombre de años murió después de cobrar su pensión en un banco de huancayo jne declara infundada apelación del apra para inscripci...,"[san, borja, minsa, lanzan, farmacia, servicio, delivery, población, vulnerable, médicos, essalud, extirpan, tumor, kilo, salvan, niño, arequipeño, años, video, hombre, años, murió, cobrar, pensión, banco, huancayo, jne, declara, infundada, apelación, apra, inscripción, listas, parlamentarias, r..."
diarioojo,octogenario muere después de cobrar su pensión en un banco de junín luciana fuster celebra su cumpleaños en yate de marc anthony el invitado de honor video mujer acusada de asesinar su esposo él se apuñaló solo video gigi mitre enfrenta son tentación hay que ser un poquito más conscientes res...,"[octogenario, muere, cobrar, pensión, banco, junín, luciana, fuster, celebra, cumpleaños, yate, marc, anthony, invitado, honor, video, mujer, acusada, asesinar, esposo, apuñaló, video, gigi, mitre, enfrenta, tentación, poquito, conscientes, responsables, video, san, borja, minsa, lanzan, farmaci..."
elcomercio_peru,keiko fujimori me llena de ilusión energía haber empezado caminar para llegar todo nuestro país línea del metro de lima nuevo horario ante ampliación del toque de queda la policía los militares de eeuu se enfrentan un enemigos internos autos antiguos en las calles limeñas sporting cristal of...,"[keiko, fujimori, llena, ilusión, energía, empezado, caminar, llegar, país, línea, metro, lima, horario, ampliación, toque, queda, policía, militares, eeuu, enfrentan, enemigos, internos, autos, antiguos, calles, limeñas, sporting, cristal, oficializó, regreso, jesús, pretell, temporada, real, m..."
larazon_pe,la premier violeta bermúdez dijo que se empezará una campaña informativa sobre la importancia de la vacunación ya que muchas personas tienen temor no creen el exministro de salud lamentó que la titular del minsa no pueda inducir al presidente estrategias mucho más claras juez del primer juzg

In [62]:
from collections import Counter

frecuency_df = []

for i in range(10):
    df = pd.DataFrame.from_dict(Counter(data_dtm.doc.loc[i]), orient='index', columns=[data_dtm.newspaper.loc[i]], dtype=np.int64).T
    frecuency_df.append(df)

frecuency_df[1]

,giulliana,loza,keiko,fujimori,realizando,visitas,norte,país,debida,autorización,...,estrictamente,esté,desactivación,terna,stephanie,frappart,vandálicos,american,airlines,ensaya
ExpresoPeru,2,2,8,9,1,1,11,52,1,4,...,1,1,1,1,1,1,1,1,1,1


In [66]:
dtm = pd.concat(frecuency_df, axis=1).fillna(0)
dtm

,conoce,actividades,virtuales,desarrollarán,aniversario,lima,transmitido,facebook,youtube,ministra,...,plata,reniec,vencido,usarse,junio,raparle,enterarse,nieta,electrónica,lograron
DiarioElPeruano,18.0,26.0,7.0,2.0,9.0,95.0,1.0,3.0,2.0,47.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ExpresoPeru,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gestionpe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
diariocorreo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
diarioojo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
elcomercio_peru,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
larazon_pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
larepublica_pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
peru21noticias,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tromepe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [67]:
dtm.to_pickle(f'{BASE_DIR}/data/interim/dtm.pkl')